In [1]:
import geopandas as gpd
import pandas as pd
import geowrangler.vector_zonal_stats as vzs

from pathlib import Path
import os

/root/miniconda3/envs/lacuna-fund/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
OUTPUT_DIR = Path("../../../data/04-output/")
RAW_DIR = Path("../../../data/02-raw/")

ADMIN_BOUNDS = Path("../../../data/01-admin-bounds/target_admin_bounds.shp")
GOOGLE_BLDG_BASE = OUTPUT_DIR / "google_v3_bldg_counts.csv"
GOOGLE_BLDGS_DIR = RAW_DIR / "google_open_buildings_v3"

## Load Admin Bounds

In [3]:
admin_gdf = gpd.read_file(ADMIN_BOUNDS)
admin_gdf.head(2)

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053..."
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039..."


In [4]:
## add brgy area
admin_gdf = admin_gdf.to_crs("epsg:3123")
admin_gdf["brgy_area_m2"] = admin_gdf.area

## Load Google Bldgs Extracted

In [5]:
# google_bldgs = pd.read_csv(GOOGLE_BLDG_BASE)
# google_bldgs.head(2)
# google_bldgs = google_bldgs.drop(columns=['Unnamed: 0'])

In [6]:
google_bldg_files = os.listdir(GOOGLE_BLDGS_DIR)
google_bldg_files

['google_lacuna_cities_bldgs_muntinlupa.geojson',
 'google_lacuna_cities_bldgs_dagupan.geojson',
 'google_lacuna_cities_bldgs_davao.geojson',
 'google_lacuna_cities_bldgs_cdo.geojson',
 'google_lacuna_cities_bldgs_palayan.geojson',
 'google_lacuna_cities_bldgs_iloilo.geojson',
 'google_lacuna_cities_bldgs_mandaue.geojson',
 'google_lacuna_cities_bldgs_zamboanga.geojson',
 'google_lacuna_cities_bldgs_navotas.geojson',
 'google_lacuna_cities_bldgs_legazpi.geojson',
 'google_lacuna_cities_bldgs_tacloban.geojson',
 'google_lacuna_cities_bldgs_mandaluyong.geojson']

In [7]:
google_gdfs = []
for bldg_file in google_bldg_files:
    gdf = gpd.read_file(GOOGLE_BLDGS_DIR / bldg_file, driver="GeoJSON")
    google_gdfs.append(gdf)

google_gdf = gpd.GeoDataFrame(pd.concat(google_gdfs, ignore_index=True))

In [8]:
google_gdf.head(2)

,city_name,barangay_psgc_code,area_in_meters,confidence,geometry
0,City of Muntinlupa,PH137603005,29.1533,0.6901,"POLYGON ((121.02198 14.36817, 121.02192 14.368..."
1,City of Muntinlupa,PH137603005,97.5825,0.7559,"POLYGON ((121.05054 14.39108, 121.05053 14.391..."


### Add binnings

In [9]:
# less than 100
is_less_100sqm = google_gdf["area_in_meters"] < 100
# 100 - 200 sqm
is_100_200sqm = (google_gdf["area_in_meters"] >= 100) & (
    google_gdf["area_in_meters"] <= 200
)
# greater than 200 sqm
is_gt_200sqm = google_gdf["area_in_meters"] > 200

google_gdf.loc[is_less_100sqm, "is_less_100sqm"] = 1
google_gdf.loc[~is_less_100sqm, "is_less_100sqm"] = 0

google_gdf.loc[is_100_200sqm, "is_100_200sqm"] = 1
google_gdf.loc[~is_100_200sqm, "is_100_200sqm"] = 0

google_gdf.loc[is_gt_200sqm, "is_gt_200sqm"] = 1
google_gdf.loc[~is_gt_200sqm, "is_gt_200sqm"] = 0

## Align to AOI

In [10]:
admin_gdf = admin_gdf.to_crs("epsg:4326")
google_gdf = google_gdf.set_crs("epsg:4326")

In [11]:
%%time
aligned_google = vzs.create_zonal_stats(
    admin_gdf,
    google_gdf,
    aggregations=[
        {"func": "count", "output": "bldg_count"},
        {
            "column": "area_in_meters",
            "func": ["sum", "mean"],
            "output": ["sum_area_in_meters", "mean_area_in_meters"],
        },
        {
            "column": "is_less_100sqm",
            "func": "sum",
            "output": "google_total_num_bldgs_lt100_sqm",
        },
        {
            "column": "is_100_200sqm",
            "func": "sum",
            "output": "google_total_num_bldgs_100_200_sqm",
        },
        {
            "column": "is_gt_200sqm",
            "func": "sum",
            "output": "google_total_num_bldgs_gt_200_sqm",
        },
    ],
)

CPU times: user 4.76 s, sys: 270 ms, total: 5.03 s
Wall time: 5.04 s


In [12]:
aligned_google

,ADM1_EN,ADM1_PCODE,ADM2_EN,ADM2_PCODE,ADM3_EN,ADM3_PCODE,ADM4_EN,ADM4_PCODE,geometry,brgy_area_m2,bldg_count,sum_area_in_meters,mean_area_in_meters,google_total_num_bldgs_lt100_sqm,google_total_num_bldgs_100_200_sqm,google_total_num_bldgs_gt_200_sqm
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,"POLYGON ((120.32742 16.05423, 120.32719 16.053...",1.020434e+06,469,18878.5810,40.252838,442.0,20.0,7.0
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,"POLYGON ((120.33380 16.03974, 120.33389 16.039...",1.042761e+06,2209,234899.6797,106.337564,1535.0,449.0,225.0
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,"POLYGON ((120.34737 16.06009, 120.34761 16.060...",3.258680e+06,6530,479063.8388,73.363528,5245.0,889.0,396.0
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,"POLYGON ((120.34054 16.04489, 120.34054 16.044...",1.811518e+05,501,81550.8829,162.776213,304.0,101.0,96.0
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,"POLYGON ((121.12250 15.58028, 121.12687 15.579...",6.335159e+06,1170,87170.2755,74.504509,929.0,202.0,39.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Namayan,PH137401018,"POLYGON ((121.02328 14.58135, 121.02309 14.581...",3.854193e+05,1314,149074.7408,113.451096,929.0,257.0,128.0
875,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Plainview,PH137401022,"POLYGON ((121.03657 14.57933, 121.03734 14.579...",1.083446e+06,4952,584873.0487,118.108451,3039.0,1195.0,718.0
876,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Navotas West,PH137503007,"MULTIPOLYGON (((120.94767 14.65256, 120.94768 ...",5.808021e+04,920,34764.5314,37.787534,868.0,48.0,4.0
877,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Tanza,PH137503014,"POLYGON ((120.91841 14.71296, 120.92356 14.708...",5.753619e+06,5553,297451.2426,53.565864,4855.0,560.0,138.0


## Add More Features

In [13]:
added_feat = aligned_google.copy()
added_feat["google_blgd_density"] = (
    added_feat["bldg_count"] / added_feat["brgy_area_m2"]
)
added_feat["google_pct_total_built_up_area"] = 100 * (
    added_feat["sum_area_in_meters"] / added_feat["brgy_area_m2"]
)

In [14]:
added_feat.columns

Index(['ADM1_EN', 'ADM1_PCODE', 'ADM2_EN', 'ADM2_PCODE', 'ADM3_EN',
       'ADM3_PCODE', 'ADM4_EN', 'ADM4_PCODE', 'geometry', 'brgy_area_m2',
       'bldg_count', 'sum_area_in_meters', 'mean_area_in_meters',
       'google_total_num_bldgs_lt100_sqm',
       'google_total_num_bldgs_100_200_sqm',
       'google_total_num_bldgs_gt_200_sqm', 'google_blgd_density',
       'google_pct_total_built_up_area'],
      dtype='object')

In [15]:
added_feat = added_feat[
    [
        "ADM4_PCODE",
        "brgy_area_m2",
        "bldg_count",
        "sum_area_in_meters",
        "mean_area_in_meters",
        "google_total_num_bldgs_lt100_sqm",
        "google_total_num_bldgs_100_200_sqm",
        "google_total_num_bldgs_gt_200_sqm",
        "google_blgd_density",
        "google_pct_total_built_up_area",
    ]
]

In [17]:
google_bldg_extract_df = pd.DataFrame(added_feat)
google_bldg_extract_df.to_csv(OUTPUT_DIR / "google_bldgs_v3_features.csv")